In [1]:
from pyspark import SparkConf, SparkContext
import csv

def extract_year(date):
    return date.split("-")[0]

if __name__ == "__main__":
    conf = SparkConf().setMaster("local").setAppName("IMDB")
    sc = SparkContext(conf=conf)
    
    #Popularnost žanrova po decenijama
    #racunata je suma
    
    mymoviedb = sc.textFile("mymoviedb.csv")
    header = mymoviedb.first()
    
    dataFilter = mymoviedb.filter(lambda line: line != header)
    
    # Extract year, popularity, and genres
    data = dataFilter.mapPartitions(lambda x: csv.reader(x)).map(lambda x: (extract_year(x[0]), float(x[3]), x[7]))
   
    # (genre (year, popularity))
    genre_popularity = data.flatMap(lambda x: [(genre.strip(), (x[0], x[1])) for genre in x[2].split(",")])
    
    # Reduce by key (genre) and year, summing up popularity scores
    genre_popularity_by_decade = genre_popularity.map(lambda x: ((x[0], x[1][0][:3] + "0s"), x[1][1])) \
        .reduceByKey(lambda a, b: a + b) \
        .sortByKey()

    output_path = "genre_popularity_by_decade.txt"
    genre_popularity_by_decade.saveAsTextFile(output_path)
        
    
    # Stop the Spark context
    sc.stop()
